In [1]:
import json
import boto3
from botocore.config import Config
from datetime import datetime

def blog_generate_using_bedrock(blog_topic: str) -> str:
    """Generates a blog using Amazon Bedrock (Llama 3 70B Instruct)."""
    try:
        # Construct the prompt for Llama 3 70B.  This model uses a simpler prompt format.
        prompt = f"Write a 200-word blog on the topic: {blog_topic}"

        body = {
            "prompt": prompt,  # Use the 'prompt' key
            "max_gen_len": 512,
            "temperature": 0.5,
            "top_p": 0.9,
        }

        bedrock = boto3.client(
            service_name='bedrock-runtime',
            region_name="us-east-1",  # Ensure this is your correct region
            config=Config(
                read_timeout=300,
                retries={'max_attempts': 3}
            )
        )

        response = bedrock.invoke_model(
            body=json.dumps(body),
            modelId="meta.llama3-8b-instruct-v1:0",  # Corrected Model ID (70B)
            contentType="application/json",
            accept="application/json"
        )

        response_content = response.get('body').read()
        response_data = json.loads(response_content)
        # print(response_data) # kept for debugging.

        # Extract the generated text (adjust based on Llama 3 70B's response format)
        blog_details = response_data['generation']
        return blog_details

    except Exception as e:
        print(f"Error generating the blog: {e}")
        return ""

def save_blog_details_s3(s3_key, s3_bucket, generated_blog):
    """Saves the generated blog to an S3 bucket."""
    s3 = boto3.client('s3')  # Initialize S3 client

    try:
        s3.put_object(Bucket=s3_bucket, Key=s3_key, Body=generated_blog)
        print("Code saved to s3")

    except Exception as e:
        print("Error when saving the code to s3")

def lambda_handler(event, context):
    """Lambda function handler."""

    body = json.loads(event['body']) # Get body from the event object
    blog_topic = body['blog_topic']   # Extract blog topic

    generated_blog = blog_generate_using_bedrock(blog_topic)

    if generated_blog:
        current_time = datetime.now().strftime('%H%M%S')
        s3_key = f"blog-output/{current_time}.txt"  #corrected
        s3_bucket_name = "aws-bedrock-course-one"  # Replace with your bucket name

        save_blog_details_s3(s3_key, s3_bucket_name, generated_blog)

    else:
        print("No blog was generated")

    return {
        'statusCode': 200,
        'body': json.dumps('Blog generation is completed')
    }

ModuleNotFoundError: No module named 'boto3'